In [ ]:
import cv2,os
data_path='data-set/'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels)) #empty dictionary

print(label_dict)
print(categories)
print(labels)

In [ ]:
img_size=48
data=[]
target=[]


for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
            #Coverting the image into gray scale
            resized=cv2.resize(gray,(img_size,img_size))
            #resizing the gray scale into 48x48, since we need a fixed common size for all the images in the dataset
            data.append(resized)
            target.append(label_dict[category])
            #appending the image and the label(categorized) into the list (dataset)

        except Exception as e:
            print('Exception:',e)
            #if any exception rasied, the exception will be printed here. And pass to the next image

In [ ]:
import numpy as np
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)
from keras.utils import np_utils
new_target=np_utils.to_categorical(target)

In [ ]:
np.save('images.npy',data)
np.save('labels.npy',new_target)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(64,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(7,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val=train_test_split(data,new_target,test_size=0.1)

In [ ]:
x_train.shape

In [9]:
#checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(x_train,y_train,epochs=20,validation_split=0.2)

Epoch 1/100
649/649 [==============================] - 88s 135ms/step - loss: 1.7694 - accuracy: 0.2774 - val_loss: 1.5760 - val_accuracy: 0.4086
Epoch 2/100
649/649 [==============================] - 89s 137ms/step - loss: 1.5557 - accuracy: 0.3933 - val_loss: 1.4578 - val_accuracy: 0.4493
Epoch 3/100
649/649 [==============================] - 90s 138ms/step - loss: 1.4558 - accuracy: 0.4286 - val_loss: 1.4084 - val_accuracy: 0.4649
Epoch 4/100
649/649 [==============================] - 90s 139ms/step - loss: 1.3769 - accuracy: 0.4706 - val_loss: 1.3452 - val_accuracy: 0.4801
Epoch 5/100
649/649 [==============================] - 90s 139ms/step - loss: 1.3372 - accuracy: 0.4869 - val_loss: 1.3421 - val_accuracy: 0.4792
Epoch 6/100
649/649 [==============================] - 91s 140ms/step - loss: 1.3037 - accuracy: 0.4985 - val_loss: 1.3217 - val_accuracy: 0.4880
Epoch 7/100
649/649 [==============================] - 91s 140ms/step - loss: 1.2701 - accuracy: 0.5115 - val_loss: 1.3134 -

Epoch 57/100
649/649 [==============================] - 90s 138ms/step - loss: 0.7245 - accuracy: 0.7262 - val_loss: 1.3881 - val_accuracy: 0.5231
Epoch 58/100
649/649 [==============================] - 90s 138ms/step - loss: 0.7396 - accuracy: 0.7215 - val_loss: 1.3770 - val_accuracy: 0.5322
Epoch 59/100
649/649 [==============================] - 90s 138ms/step - loss: 0.7243 - accuracy: 0.7319 - val_loss: 1.3949 - val_accuracy: 0.5239
Epoch 60/100
649/649 [==============================] - 90s 138ms/step - loss: 0.7219 - accuracy: 0.7322 - val_loss: 1.4421 - val_accuracy: 0.5253
Epoch 61/100
649/649 [==============================] - 89s 138ms/step - loss: 0.7090 - accuracy: 0.7346 - val_loss: 1.4134 - val_accuracy: 0.5297
Epoch 62/100
649/649 [==============================] - 90s 138ms/step - loss: 0.7217 - accuracy: 0.7334 - val_loss: 1.4500 - val_accuracy: 0.5330
Epoch 63/100
649/649 [==============================] - 90s 138ms/step - loss: 0.7016 - accuracy: 0.7340 - val_loss: 1

In [ ]:
 model.save('model.h5')

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train Accuracy', 'Validation Accuracy'], loc='lower left')
plt.savefig("Cnn_acc")
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training Loss', 'Validation Loss'], loc='upper right')
plt.savefig("Cnn_loss")
plt.show()

In [ ]:
import itertools


%matplotlib inline

In [ ]:
Y_val_pred = model.predict(x_val)

In [ ]:

from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, classes,
                          normalize=True,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=55)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')

    plt.tight_layout()
    plt.grid(False)
    plt.savefig("Cnn_conf_matrix.png")
cm = confusion_matrix(np.argmax(y_val, axis=1), np.argmax(Y_val_pred, axis=1))

cm_plot_label =['Angry', 'Disgust','Fear','Happy','neutral','Sad','Surprise']
plot_confusion_matrix(cm, cm_plot_label, title ='Confusion Metrix')